In [3]:
from cmath import isnan
import os
from random import random
import time
import pandas as pd
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.common.exceptions import NoSuchElementException
from csv import writer

# read the data from a file
data = pd.read_csv(os.getcwd().replace('code/old_test_code', '') + "/data/cleaned_prof_data_v4.csv")

# initialize web driver for selenium
options = webdriver.ChromeOptions()
options.add_argument('--disable-blink-features=AutomationControlled')
driver = webdriver.Chrome(ChromeDriverManager().install(), options=options)
driver.implicitly_wait(random() * 10)


def findMaxCitations(web_elements: list):
    indOfMax = 0
    maxVal = 0
    for i in range(len(web_elements)):
        val = int(web_elements[i].text.split(" - ")[2].split(" ")[2])
        if val > maxVal:
            maxVal = val
            indOfMax = i
        else:
            continue
    return web_elements[indOfMax]


# loop through and find all professors with wrong / no google scholar ID
professors = []
professorExists = True
for i in range(data.shape[0]):

    #initialize values 
    scholar_id = data.iat[i,3],
    citations = data.iat[i,4], 
    h_index  = data.iat[i,5]

    if(data.iat[i, 3] == "NOSCHOLARPAGE" or data.iat[i,3] == "#NAME?" or isnan(data.iat[i,4]) or isnan(data.iat[i,5])):
        print("Name: " + data.iat[i, 0] + " | " + "University: " +
              data.iat[i, 1] + " | " + "id: " + data.iat[i, 3])

        # launch URL
        driver.get("https://scholar.google.com")

        # identify search box
        box = driver.find_element(By.NAME, "q")

        # enter search text
        box.send_keys(data.iat[i, 0] +  " " + data.iat[i,1])

        # make search
        box.send_keys(Keys.ENTER)

        # in the case that the author pops up as a recommended option
        try:
            link = driver.find_element(
                By.XPATH, "//*[@id='gs_res_ccl_mid']/div[1]/table/tbody/tr/td[2]/h4/a")
            link.click()
            time.sleep(random() * 3)
        except:
            try:
                # in the case there are multiple options in the recommended table
                list_of_elements = driver.find_element(
                    By.XPATH, "//*[@id='gs_res_ccl_mid']/div[1]/table/tbody/tr/td[2]").find_elements(By.TAG_NAME, "div")
                print("case 2 was executed")
                max_element = findMaxCitations(list_of_elements)
                max_element.find_element(By.TAG_NAME, "a").click()
                time.sleep(random() * 5)
            except:
                # otherwise loop through all the divs and find the correct professor
                elements_list = driver.find_element(
                    By.XPATH, "//*[@id='gs_res_ccl_mid']/div[1]").find_element(By.CLASS_NAME, "gs_a").find_elements(By.TAG_NAME, "a")

                print(elements_list)

                for element in elements_list:  # looping through the number of professors in the link
                    try:
                        element.find_element(By.TAG_NAME, "b")
                        element.click()
                        break
                    except NoSuchElementException:
                        professorExists = False
                        continue  # in the case that the element is not found
                # give time for html / css / js to load
                time.sleep(random() * 5)

                # retrieve citation and h-index for that prof
        if professorExists:

            scholar_id = driver.current_url.split("=")[1].split("&")[0]
            citation = driver.find_element(
                By.XPATH, "//*[@id='gsc_rsb_st']/tbody/tr[1]/td[2]").text
            h_index = driver.find_element(
                By.XPATH, "//*[@id='gsc_rsb_st']/tbody/tr[2]/td[2]").text

            time.sleep(random() * 5)

        else:
            scholar_id = None
            citation = None 
            h_index = None 
            continue
    else:
        continue
    
    professors.append({ 
            "Name": data.iat[0],
            "affiliation": data.iat[1],
            "homepage": data.iat[2],
            "scholar-id": scholar_id,
            "citations": citation, 
            "h-index": h_index
    })

print(len(professors))
df = pd.DataFrame(professors)
df.to_csv("cleaned_prof_data_v5.csv")



[WDM] - ====== WebDriver manager ======


2022-07-15 17:41:37,546 INFO ====== WebDriver manager ======


[WDM] - Current google-chrome version is 103.0.5060


2022-07-15 17:41:37,624 INFO Current google-chrome version is 103.0.5060


[WDM] - Get LATEST chromedriver version for 103.0.5060 google-chrome


...

/var/folders/sw/rc481v691q5883ld6bm6jzq80000gn/T/ipykernel_25093/3188425993.py:19: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(ChromeDriverManager().install(), options=options)


Name: P. Vassalos | University: AUEB | id: NOSCHOLARPAGE
[<selenium.webdriver.remote.webelement.WebElement (session="d7c97efce88d1caeb13f5448cfbfbc1c", element="eae30933-896e-426b-91b7-1ec65a4dca00")>, <selenium.webdriver.remote.webelement.WebElement (session="d7c97efce88d1caeb13f5448cfbfbc1c", element="ea2b9082-bb3b-460f-814c-8c5135c2a1d0")>]
Name: Eugénie Foustoucos | University: AUEB | id: t1TycUYAAAAJ
Name: Evangelos Markakis | University: AUEB | id: NOSCHOLARPAGE
Name: Nicos Malevris | University: AUEB | id: NOSCHOLARPAGE
[<selenium.webdriver.remote.webelement.WebElement (session="d7c97efce88d1caeb13f5448cfbfbc1c", element="8a01863b-fa53-434f-8a98-04b087a04a11")>, <selenium.webdriver.remote.webelement.WebElement (session="d7c97efce88d1caeb13f5448cfbfbc1c", element="cb06ef11-1653-400e-875b-4c3245812c90")>, <selenium.webdriver.remote.webelement.WebElement (session="d7c97efce88d1caeb13f5448cfbfbc1c", element="ee894949-d7d5-402b-b2bd-8dfd9710d518")>, <selenium.webdriver.remote.webelem

Name: George M. Turkiyyah | University: American University of Beirut | id: NOSCHOLARPAGE
[<selenium.webdriver.remote.webelement.WebElement (session="d7c97efce88d1caeb13f5448cfbfbc1c", element="f2cb2d1a-e100-4100-a9c5-4ebe264a08be")>, <selenium.webdriver.remote.webelement.WebElement (session="d7c97efce88d1caeb13f5448cfbfbc1c", element="f83d1993-7dfa-4275-a410-95629d81fdfe")>, <selenium.webdriver.remote.webelement.WebElement (session="d7c97efce88d1caeb13f5448cfbfbc1c", element="202ec2d6-b1d5-424f-9922-77f393397c55")>]
Name: Imad Elhajj | University: American University of Beirut | id: GtBBM68AAAAJ
Name: Dana Shapira | University: Ariel University | id: Vt744hgAAAAJ
Name: Dan Ophir | University: Ariel University | id: NOSCHOLARPAGE
[]
Name: Chaowei Xiao | University: Arizona State University | id: Juoqtj8AAAA
[<selenium.webdriver.remote.webelement.WebElement (session="d7c97efce88d1caeb13f5448cfbfbc1c", element="97c9fa3c-d721-45e5-945e-4bd0635e0ea8")>, <selenium.webdriver.remote.webelemen

Name: Rizwan Parveen | University: BITS Pilani-Goa | id: NOSCHOLARPAGE
[<selenium.webdriver.remote.webelement.WebElement (session="d7c97efce88d1caeb13f5448cfbfbc1c", element="8a608c8b-204a-44da-87ed-69c7f6e8cd16")>]
Name: Swati Agarwal | University: BITS Pilani-Goa | id: #NAME?
Name: A. Bhaskar | University: BITS Pilani-Goa | id: WNrQSakAAAAJ
[<selenium.webdriver.remote.webelement.WebElement (session="d7c97efce88d1caeb13f5448cfbfbc1c", element="6c9e8aef-9d1b-465c-bce9-fe1882a48194")>, <selenium.webdriver.remote.webelement.WebElement (session="d7c97efce88d1caeb13f5448cfbfbc1c", element="33643327-4db9-46bc-b854-41a1a479f2d2")>, <selenium.webdriver.remote.webelement.WebElement (session="d7c97efce88d1caeb13f5448cfbfbc1c", element="9680a673-f6ab-491a-8120-4d9bf9521fbc")>]
Name: Muhammad Masroor Ali | University: BUET | id: N8d3dk4AAAAJ
[<selenium.webdriver.remote.webelement.WebElement (session="d7c97efce88d1caeb13f5448cfbfbc1c", element="116feb35-1128-4c74-b176-8a0bbcf96bbb")>]
Name: Abul K

NoSuchElementException: Message: no such element: Unable to locate element: {"method":"xpath","selector":"//*[@id='gs_res_ccl_mid']/div[1]"}
  (Session info: chrome=103.0.5060.114)
Stacktrace:
0   chromedriver                        0x0000000101297079 chromedriver + 4444281
1   chromedriver                        0x0000000101223403 chromedriver + 3970051
2   chromedriver                        0x0000000100ebe038 chromedriver + 409656
3   chromedriver                        0x0000000100ef42fd chromedriver + 631549
4   chromedriver                        0x0000000100ef4581 chromedriver + 632193
5   chromedriver                        0x0000000100f264f4 chromedriver + 836852
6   chromedriver                        0x0000000100f117bd chromedriver + 751549
7   chromedriver                        0x0000000100f24221 chromedriver + 827937
8   chromedriver                        0x0000000100f11683 chromedriver + 751235
9   chromedriver                        0x0000000100ee7a45 chromedriver + 58